## 1. Configuration

Set the paths and flags for the modeling process below. 
- `TRAIN_NEW_MODELS`: Set to `True` to run training and tuning. Set to `False` to load existing models.
- `INPUT_DIR`: The directory where your source data CSV is located.
- `DATAFRAME_NAME`: The name of your CSV file (without the `.csv` extension).
- `OUTPUT_DIR_WINDOWS`: The root folder on your Windows drive where model pipelines will be saved. The path is written in WSL format (e.g., `/mnt/d/` for the `D:` drive).

In [22]:
import os

# Set to True to run the full training and tuning process.
# Set to False to load pre-existing models from the output directory.
TRAIN_NEW_MODELS = True

# PATHS
INPUT_DIR = os.path.join('..', '..', 'data', 'upsampled')
DATAFRAME_NAME = 'mean_df' # Name of the .csv file without the extension

# Output directory for saving model pipelines (WSL format for Windows D: drive)
# This path corresponds to D:\ML_Pipelines in Windows
OUTPUT_ROOT_DIR_WINDOWS = '/mnt/d/EMEWS_ML_Pipelines_Output/best_features_timeseries'
DATAFRAME_SPECIFIC_PATH = os.path.join(OUTPUT_ROOT_DIR_WINDOWS, DATAFRAME_NAME)
BASE_MODEL_PATH = os.path.join(DATAFRAME_SPECIFIC_PATH, 'base_models')
TUNED_MODEL_PATH = os.path.join(DATAFRAME_SPECIFIC_PATH, 'tuned_models')

# Create directories if they don't exist
if TRAIN_NEW_MODELS:
    print(f"Creating directory for base models: {BASE_MODEL_PATH}")
    os.makedirs(BASE_MODEL_PATH, exist_ok=True)
    print(f"Creating directory for tuned models: {TUNED_MODEL_PATH}")
    os.makedirs(TUNED_MODEL_PATH, exist_ok=True)
else:
    print("TRAIN_NEW_MODELS is False. Will attempt to load existing models.")

Creating directory for base models: /mnt/d/EMEWS_ML_Pipelines_Output/best_features_timeseries/mean_df/base_models
Creating directory for tuned models: /mnt/d/EMEWS_ML_Pipelines_Output/best_features_timeseries/mean_df/tuned_models


## 2. Setup and Data Loading

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pycaret.time_series import *

In [24]:
# features_to_load = ['zone_a_mwr_patients',
#  'zone_a_mwr_cat_3',
#  'zone_a_mwr_cat_4',
#  'zone_a_mwr_sets_of_emews',
#  'zone_a_mwr_deescalations',
#  'zone_a_mwr_escalations',
#  'zone_a__patients',
#  'zone_a__cat_2',
#  'zone_a__sets_of_emews',
#  'zone_b/c_patients',
#  'zone_b/c_cat_2',
#  'zone_b/c_sets_of_emews',
#  'total_number_of_emews',
#  'total_number_of_patients']

In [25]:
features_to_load = ['zone_a_mwr_patients',
 'zone_a_mwr_cat_3',
 'zone_a_mwr_cat_4',
 'zone_a_mwr_sets_of_emews',
 'zone_a_mwr_deescalations',
 'zone_a_mwr_escalations',
 'zone_a__patients',
 'zone_a__cat_2',
 'zone_a__sets_of_emews',
 'zone_b/c_patients',
 'zone_b/c_cat_2',
 'zone_b/c_sets_of_emews',
 'total_number_of_emews',
 'total_number_of_patients']

In [26]:
df = pd.read_csv(os.path.join(INPUT_DIR, f'{DATAFRAME_NAME}.csv'),  parse_dates=['date'])
df.set_index('date', inplace=True)

## 3. Pycaret Setup

In [27]:
TARGET_COLUMN = 'total_number_of_patients'
df = df[features_to_load]

In [28]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=60, target=TARGET_COLUMN, session_id=123);

,Description,Value
0,session_id,123
1,Target,total_number_of_patients
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(618, 14)"
5,Transformed data shape,"(618, 14)"
6,Transformed train set shape,"(558, 14)"
7,Transformed test set shape,"(60, 14)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


## 4. Model Training or Loading

Based on the `TRAIN_NEW_MODELS` flag, this section will either train and save new models or load existing ones.

In [29]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [30]:
created_models = {}
tuned_models = {}

if TRAIN_NEW_MODELS:
    print("Starting Model Training and Tuning")
    
    # Step 1: Compare base models
    # exp.compare_models()
    # best_models_df = exp.pull()
    # model_names_to_process = best_models_df.index[:5].tolist()
    model_names_to_process = ['arima', 'auto_arima']
    # Step 2: Create, save, and tune base models
    for model_name in model_names_to_process:
        print(f'\n--- Processing Model: {model_name} ---')
        
        # Create base model
        print(f'Creating base model: {model_name}')
        base_model = exp.create_model(model_name, verbose=False)
        created_models[model_name] = base_model
        
        # Save base model pipeline
        save_path_base = os.path.join(BASE_MODEL_PATH, model_name)
        print(f'Saving base model to: {save_path_base}')
        exp.save_model(base_model, save_path_base, model_only=True)
        
        # Tune model
        print(f'Tuning model: {model_name}')

        if model_name == 'auto_arima':
            print('Auto Arima model already tuned - skipping tuning step.')
            tuned_model = base_model
        else:
            tuned_model = exp.tune_model(base_model)
        tuned_models[model_name] = tuned_model
        
        # Save tuned model pipeline
        save_path_tuned = os.path.join(TUNED_MODEL_PATH, model_name)
        print(f'Saving tuned model to: {save_path_tuned}')
        exp.save_model(tuned_model, save_path_tuned, model_only=True)

else:
    print("--- Loading Existing Models ---")
    # Load base and tuned models if they exist
    if os.path.exists(BASE_MODEL_PATH):
        model_names_to_process = [os.path.splitext(f)[0] for f in os.listdir(BASE_MODEL_PATH) if f.endswith('.pkl')]
        print(f"Found models in {BASE_MODEL_PATH}: {model_names_to_process}")
    else:
        print(f"ERROR: Base model directory not found at {BASE_MODEL_PATH}. Cannot load models.")
        model_names_to_process = []
    
    for name in model_names_to_process:
        base_path = os.path.join(BASE_MODEL_PATH, name)
        tuned_path = os.path.join(TUNED_MODEL_PATH, name)
        
        # Load Base Model
        if os.path.exists(f'{base_path}.pkl'):
            print(f'Loading base model: {name} from {base_path}')
            created_models[name] = exp.load_model(base_path, verbose=False)
        else:
            print(f'WARNING: Base model for {name} not found at {base_path}.pkl')
            
        # Load Tuned Model
        if os.path.exists(f'{tuned_path}.pkl'):
            print(f'Loading tuned model: {name} from {tuned_path}')
            tuned_models[name] = exp.load_model(tuned_path, verbose=False)
        else:
            print(f'WARNING: Tuned model for {name} not found at {tuned_path}.pkl')

print("\nModel processing complete.")
print(f"\nBase models available: {list(created_models.keys())}")
print(f"Tuned models available: {list(tuned_models.keys())}")

Starting Model Training and Tuning

--- Processing Model: arima ---
Creating base model: arima
Saving base model to: /mnt/d/EMEWS_ML_Pipelines_Output/best_features_timeseries/mean_df/base_models/arima
Model Successfully Saved
Tuning model: arima


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-07-10 12:00,0.1621,0.1575,2.9955,3.8589,0.1119,0.1087,0.9548
1,2024-08-09 12:00,0.2427,0.3163,4.5887,7.9036,0.1239,0.1551,0.8753
2,2024-09-08 12:00,0.2295,0.2426,4.4891,6.2267,0.0978,0.1063,0.9151
Mean,NaT,0.2114,0.2388,4.0244,5.9964,0.1112,0.1234,0.9151
SD,NaT,0.0353,0.0649,0.7287,1.6593,0.0107,0.0224,0.0325


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.4min finished


Saving tuned model to: /mnt/d/EMEWS_ML_Pipelines_Output/best_features_timeseries/mean_df/tuned_models/arima
Model Successfully Saved

--- Processing Model: auto_arima ---
Creating base model: auto_arima
Saving base model to: /mnt/d/EMEWS_ML_Pipelines_Output/best_features_timeseries/mean_df/base_models/auto_arima
Model Successfully Saved
Tuning model: auto_arima
Auto Arima model already tuned - skipping tuning step.
Saving tuned model to: /mnt/d/EMEWS_ML_Pipelines_Output/best_features_timeseries/mean_df/tuned_models/auto_arima
Model Successfully Saved

Model processing complete.

Base models available: ['arima', 'auto_arima']
Tuned models available: ['arima', 'auto_arima']


## 5. Custom Metrics and Final Predictions
This section defines and adds custom metrics for evaluating predictions on the hold-out set, then generates and saves the final performance metrics to an Excel file.

In [31]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def r2_rounded(y_true, y_pred):
    """Calculates R2 score after rounding predictions to the nearest whole number."""
    return r2_score(y_true, np.round(y_pred))

def rmse_rounded(y_true, y_pred):
    """Calculates RMSE after rounding predictions to the nearest whole number."""
    return np.sqrt(mean_squared_error(y_true, np.round(y_pred)))

def r2_ceil(y_true, y_pred):
    """Calculates R2 score after ceiling predictions to the nearest whole number."""
    return r2_score(y_true, np.ceil(y_pred))

def rmse_ceil(y_true, y_pred):
    """Calculates RMSE after ceiling predictions to the nearest whole number."""
    return np.sqrt(mean_squared_error(y_true, np.ceil(y_pred)))

def mae_rounded(y_true, y_pred):
    """Calculates MAE after rounding predictions to the nearest whole number."""
    return mean_absolute_error(y_true, np.round(y_pred))

def mae_ceil(y_true, y_pred):
    """Calculates MAE after ceiling predictions to the nearest whole number."""
    return mean_absolute_error(y_true, np.ceil(y_pred))

In [32]:
try:
    exp.add_metric('R2_Rounded', 'R2_RND', r2_rounded, greater_is_better=True)
    exp.add_metric('RMSE_Rounded', 'RMSE_RND', rmse_rounded, greater_is_better=False)
    exp.add_metric('MAE_Rounded', 'MAE_RND', mae_rounded, greater_is_better=False)
    exp.add_metric('R2_Ceil', 'R2_CEIL', r2_ceil, greater_is_better=True)
    exp.add_metric('RMSE_Ceil', 'RMSE_CEIL', rmse_ceil, greater_is_better=False)
    exp.add_metric('MAE_Ceil', 'MAE_CEIL', mae_ceil, greater_is_better=False)
except ValueError:
    print("Metrics may have already been added in this session.")

In [33]:
# Generate predictions for base models
holdout_predictions_metric = {}
if not created_models:
    print("No base models available to make predictions.")
else:
    for model_name, model_object in created_models.items():
        print(f"Generating predictions for base model: {model_name}")
        exp.predict_model(model_object, verbose=False)
        holdout_predictions_metric[model_name] = exp.pull()

# Generate predictions for tuned models
tuning_predictions_metric = {}
if not tuned_models:
    print("No tuned models available to make predictions.")
else:
    for model_name, model_object in tuned_models.items():
        print(f"Generating predictions for tuned model: {model_name}")
        exp.predict_model(model_object, verbose=False)
        tuning_predictions_metric[model_name] = exp.pull()

Generating predictions for base model: arima
Generating predictions for base model: auto_arima
Generating predictions for tuned model: arima
Generating predictions for tuned model: auto_arima


In [34]:
output_excel_path = os.path.join(DATAFRAME_SPECIFIC_PATH, 'model_performance_metrics.xlsx')
print(f"Saving performance metrics to: {output_excel_path}")

with pd.ExcelWriter(output_excel_path) as writer:
    # --- Process and Save Base Model Metrics ---
    if holdout_predictions_metric:
        list_of_metric_dfs_base = []
        for model_name, metrics_df in holdout_predictions_metric.items():
            list_of_metric_dfs_base.append(metrics_df)
        
        results_df_base = pd.concat(list_of_metric_dfs_base, ignore_index=True).sort_values('R2', ascending=False)
        print("\n--- Base Model Holdout Predictions ---")
        print(results_df_base.to_string())
        results_df_base.to_excel(writer, sheet_name='Base Model Metrics', index=False)
    else:
        print("\nNo base model metrics to save.")

    # --- Process and Save Tuned Model Metrics ---
    if tuning_predictions_metric:
        list_of_metric_dfs_tuned = []
        for model_name, metrics_df in tuning_predictions_metric.items():
            list_of_metric_dfs_tuned.append(metrics_df)
            
        results_df_tuned = pd.concat(list_of_metric_dfs_tuned, ignore_index=True).sort_values('R2', ascending=False)
        print("\n--- Tuned Model Holdout Predictions ---")
        print(results_df_tuned.to_string())
        results_df_tuned.to_excel(writer, sheet_name='Tuned Model Metrics', index=False)
    else:
        print("\nNo tuned model metrics to save.")

Saving performance metrics to: /mnt/d/EMEWS_ML_Pipelines_Output/best_features_timeseries/mean_df/model_performance_metrics.xlsx

--- Base Model Holdout Predictions ---
        Model    MASE   RMSSE     MAE    RMSE    MAPE   SMAPE      R2  R2_RND  RMSE_RND  MAE_RND  R2_CEIL  RMSE_CEIL  MAE_CEIL
1  Auto ARIMA  0.1581  0.1668  3.1761  4.3907  0.0768  0.0734  0.9538  0.9534    4.4083   3.2333   0.9503     4.5552      3.25
0       ARIMA  0.1854  0.1852  3.7248  4.8769  0.0742  0.0736  0.9430  0.9428    4.8871   3.7167   0.9440     4.8356      3.65

--- Tuned Model Holdout Predictions ---
        Model    MASE   RMSSE     MAE    RMSE    MAPE   SMAPE      R2  R2_RND  RMSE_RND  MAE_RND  R2_CEIL  RMSE_CEIL  MAE_CEIL
0       ARIMA  0.1638  0.1667  3.2920  4.3882  0.0659  0.0676  0.9538  0.9523    4.4628   3.3167   0.9555     4.3070    3.1833
1  Auto ARIMA  0.1581  0.1668  3.1761  4.3907  0.0768  0.0734  0.9538  0.9534    4.4083   3.2333   0.9503     4.5552    3.2500


In [37]:
tuned_models['arima'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  558
Model:               SARIMAX(0, 0, 1)   Log Likelihood               -1745.122
Date:                Wed, 27 Aug 2025   AIC                           3520.243
Time:                        19:27:22   BIC                           3585.109
Sample:                    01-04-2024   HQIC                          3545.576
                         - 10-08-2024                                         
Covariance Type:                  opg                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
zone_a_mwr_patients          0.9001      0.307      2.931      0.003       0.298       1.502
zone_a_mwr_cat_3            -0.1025      0.307     -0.334      0.738      -0.704       0.499
zone_a_mwr_cat_4            -0.2866      0.315     -0.909      0.364      -0.905       0.332
zone_a_mwr_sets_of_emews    -0.2717      0.018    -14.686      0.000      -0.308      -0.235
zone_a_mwr_deescalations    -0.0230      0.048     -0.475      0.635      -0.118       0.072
zone_a_mwr_escalations      -0.1311      0.231     -0.568      0.570      -0.584       0.322
zone_a__patients             1.0403      0.071     14.686      0.000       0.901       1.179
zone_a__cat_2               -0.0270      0.071     -0.383      0.702      -0.165       0.111
zone_a__sets_of_emews       -0.3218      0.013    -24.668      0.000      -0.347      -0.296
zone_b/c_patients            1.1139      0.204      5.449      0.000       0.713       1.515
zone_b/c_cat_2              -0.1162      0.358     -0.325      0.745      -0.817       0.585
zone_b/c_sets_of_emews      -0.2967      0.024    -12.613      0.000      -0.343      -0.251
total_number_of_emews        0.3264      0.006     58.944      0.000       0.316       0.337
ma.L1                        0.0584      0.061      0.954      0.340      -0.062       0.178
sigma2                      30.5385      1.294     23.603      0.000      28.003      33.074
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):             23466.72
Prob(Q):                              0.87   Prob(JB):                         0.00
Heteroskedasticity (H):               0.78   Skew:                             0.75
Prob(H) (two-sided):                  0.10   Kurtosis:                        34.73
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [35]:
tuned_models['auto_arima'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  558
Model:               SARIMAX(1, 1, 2)   Log Likelihood               -1726.086
Date:                Wed, 27 Aug 2025   AIC                           3486.172
Time:                        19:14:49   BIC                           3559.655
Sample:                    01-04-2024   HQIC                          3514.872
                         - 10-08-2024                                         
Covariance Type:                  opg                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
zone_a_mwr_patients          0.9097      0.407      2.237      0.025       0.113       1.707
zone_a_mwr_cat_3            -0.1168      0.408     -0.286      0.775      -0.917       0.683
zone_a_mwr_cat_4            -0.2424      0.413     -0.587      0.557      -1.051       0.567
zone_a_mwr_sets_of_emews    -0.2669      0.016    -16.470      0.000      -0.299      -0.235
zone_a_mwr_deescalations     0.0118      0.049      0.239      0.811      -0.085       0.108
zone_a_mwr_escalations      -0.2113      0.228     -0.928      0.354      -0.658       0.235
zone_a__patients             1.0489      0.061     17.224      0.000       0.930       1.168
zone_a__cat_2               -0.0622      0.065     -0.958      0.338      -0.190       0.065
zone_a__sets_of_emews       -0.3170      0.014    -22.795      0.000      -0.344      -0.290
zone_b/c_patients            0.9873      0.225      4.382      0.000       0.546       1.429
zone_b/c_cat_2              -0.1816      0.368     -0.493      0.622      -0.903       0.540
zone_b/c_sets_of_emews      -0.2910      0.027    -10.836      0.000      -0.344      -0.238
total_number_of_emews        0.3161      0.006     50.518      0.000       0.304       0.328
ar.L1                       -0.8018      0.222     -3.607      0.000      -1.238      -0.366
ma.L1                       -0.2052      0.251     -0.818      0.413      -0.697       0.286
ma.L2                       -0.7125      0.248     -2.872      0.004      -1.199      -0.226
sigma2                      28.5901      1.071     26.689      0.000      26.491      30.690
===================================================================================
Ljung-Box (L1) (Q):                   0.90   Jarque-Bera (JB):             23354.68
Prob(Q):                              0.34   Prob(JB):                         0.00
Heteroskedasticity (H):               0.82   Skew:                             1.00
Prob(H) (two-sided):                  0.19   Kurtosis:                        34.66
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [36]:
with open("sarimax_summary.html", "w") as f:
    f.write(tuned_models['auto_arima'].summary().as_html())